In [2]:
import math
# Importing standard Qiskit libraries and configuring account
import matplotlib.pyplot as plt
import numpy as np
from math import pi
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
# Loading your IBM Q account(s)
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute
from qiskit.tools.visualization import circuit_drawer
from qiskit.quantum_info import state_fidelity
from qiskit import BasicAer
IBMQ.load_account() 
backend = BasicAer.get_backend('unitary_simulator')
provider = IBMQ.get_provider(hub='ibm-q-research')



#%matplotlib inline
# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit, execute, Aer
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit,  execute, BasicAer, Aer

#error mitigation toolbox
#from qiskit.ignis.mitigation.measurement import complete_meas_cal, CompleteMeasFitter

ibmqfactory.load_account:WARNING:2022-08-29 02:49:39,541: Credentials are already in use. The existing account in the session will be replaced.


In [3]:
import pandas as pd
filename = '3outcome_POVM.csv'
filename2 = '3outcome_est.csv'
data = pd.read_csv(filename, sep=",", header=None)
data = data.applymap(lambda s: np.complex(s.replace('i', 'j'))).values

data2 = pd.read_csv(filename2, sep=",", header=None)
#data2 = data2.applymap(lambda s: ).values


print(data[[0]])

datadag=data.conj().T
print(datadag[[0]])

data2=np.array(data2)
print(data2)

[[0.4993+0.026j  0.4968-0.0563j 0.4086-0.2881j 0.    +0.j
  0.    +0.j     0.    +0.j     0.    +0.j     0.0574+0.4967j]]
[[ 0.4993-0.026j  -0.2887-0.0006j -0.2887-0.0006j  0.2882+0.0162j
  -0.2887-0.0006j  0.2882+0.0162j  0.2882+0.0162j -0.497 -0.0551j]]
[[-0.1338  0.1338 -2.4681  0.7817 -0.7817  0.7156 -0.7156  2.4681]
 [-2.468   2.468   0.1338  0.7157 -0.7157 -0.7819  0.7819 -0.1338]]


/tmp/ipykernel_138/346370027.py:5: DeprecationWarning: `np.complex` is a deprecated alias for the builtin `complex`. To silence this warning, use `complex` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.complex128` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  data = data.applymap(lambda s: np.complex(s.replace('i', 'j'))).values


In [4]:
#name I decided to give our 'quantum computer' in simulations
cname='ibmq_edendork'


In [5]:

from scipy import stats
from scipy.optimize import fsolve

In [7]:
#data analysis
#two noise strings, one for large noise one for small
noisestring=str('aug22_noiseth0n0.001')
#noisestring=str('aug22_noiseth0n0.005')
th=0

#first load training data to get the slopes and intercepts again

F = np.load(str(cname+str("_LR")+str("/")+noisestring+str("_3Q_est_X")+str("/")+str("training_data")+str(".npz")))
FY = np.load(str(cname+str("_LR")+str("/")+noisestring+str("_3Q_est_Y")+str("/")+str("training_data")+str(".npz")))


a=F['arr_0']
aY=FY['arr_0']
noisyx=np.transpose(a)[0]
truex=np.transpose(a)[1]
noisyy=np.transpose(aY)[0]
truey=np.transpose(aY)[1]


res3QX = stats.linregress(np.array(noisyx),np.array(truex))
res3QY = stats.linregress(np.array(noisyy),np.array(truey))

#parameters from above
cx=res3QX.intercept
mx=res3QX.slope
cy=res3QY.intercept
my=res3QY.slope

#then load data data to get the total number of counts in each
FN = np.load(str(cname+str("_LR")+str("/")+noisestring+str("_3Q_nvec")+str("/")+str("nvec_data")+str(".npz")))
a=FN['arr_0']



n000T=0
n001T=0
n010T=0
n011T=0
n100T=0
n101T=0
n110T=0
n111T=0


#real mse obtained here
msereal=[]
shotsin=341
for ii in range(1000):
    n000=a[ii][0]
    n000T=n000T+a[ii][0]
    n001=a[ii][1]
    n001T=n001T+a[ii][1]
    n010=a[ii][2]
    n010T=n010T+a[ii][2]
    n011=a[ii][3]
    n011T=n011T+a[ii][3]
    n100=a[ii][4]
    n100T=n100T+a[ii][4]
    n101=a[ii][5]
    n101T=n101T+a[ii][5]
    n110=a[ii][6]
    n110T=n110T+a[ii][6]
    n111=a[ii][7]
    n111T=n111T+a[ii][7]
    estx=(n000*data2[0][0]+n001*data2[0][1]+n010*data2[0][2]+n011*data2[0][3]+n100*data2[0][4]+n101*data2[0][5]+n110*data2[0][6]+n111*data2[0][7])/shotsin
    esty=(n000*data2[1][0]+n001*data2[1][1]+n010*data2[1][2]+n011*data2[1][3]+n100*data2[1][4]+n101*data2[1][5]+n110*data2[1][6]+n111*data2[1][7])/shotsin
    estyA=esty*my+cy
    estxA=estx*mx+cx
    msereal.append(estxA**2+estyA**2)

ntot=n000T+n001T+n010T+n011T+n100T+n101T+n110T+n111T

p000=n000T/ntot
p001=n001T/ntot
p010=n010T/ntot
p011=n011T/ntot
p100=n100T/ntot
p101=n101T/ntot
p110=n110T/ntot
p111=n111T/ntot


mseT=[]

for hh in range(500):
    mseint=[]
    if ii%100==0:
        print('Bootstrap run #',ii)
    for kk in range(1000):

        rv= np.random.multinomial(shotsin, [p000,p001,p010,p011,p100,p101,p110,p111], size=1)
        n000=rv[0][0]
        n001=rv[0][1]
        n010=rv[0][2]
        n011=rv[0][3]
        n100=rv[0][4]
        n101=rv[0][5]
        n110=rv[0][6]
        n111=rv[0][7]
        
        
        estx=(n000*data2[0][0]+n001*data2[0][1]+n010*data2[0][2]+n011*data2[0][3]+n100*data2[0][4]+n101*data2[0][5]+n110*data2[0][6]+n111*data2[0][7])/shotsin
        esty=(n000*data2[1][0]+n001*data2[1][1]+n010*data2[1][2]+n011*data2[1][3]+n100*data2[1][4]+n101*data2[1][5]+n110*data2[1][6]+n111*data2[1][7])/shotsin
        estyA=esty*my+cy
        estxA=estx*mx+cx
        mseint.append(estxA**2+estyA**2)
        
    mseT.append(np.mean(mseint))
    
print('Real MSE = ',np.mean(msereal))
print('Bootstrapped MSE = ',np.mean(mseT),'+/-',np.std(mseT))

Real MSE =  0.013500184400387635
Bootstrapped MSE =  0.013557208853820796 +/- 0.00042287346761237595
